<a href="https://colab.research.google.com/github/willianpsantos/gpu-processing/blob/main/gpu_formula_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 553, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 553 (delta 179), reused 147 (delta 100), pack-reused 269 (from 1)
Receiving objects: 100% (553/553), 178.44 KiB | 16.22 MiB/s, done.
Resolving deltas: 100% (281/281), done.


In [2]:
!pip3 install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 68.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
!python3 /content/rapidsai-csp-utils/colab/pip-install.py

Installing RAPIDS remaining 24.10.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.7/567.7 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 129.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.5/915.5 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 134.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.9/196.9 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 836.6/836.6 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 138.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.7/722.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.0/916.0 kB 47.1 MB/s e

In [5]:
import cudf as pd
from numba import cuda

df = pd.read_csv('/content/gpu_data/MAIN_TABLE_TO_PROCESS.csv', ';')

In [12]:
df_additional_measures = pd.read_csv('/content/gpu_data/ADDITIONAL_MEASURES.csv', ';')
df_additional_tw_measures = pd.read_csv('/content/gpu_data/ADDITIONAL_TOWERCO_MEASURES.csv', ';')
df_exchange_rates = pd.read_csv('/content/gpu_data/EXCHANGE_RATE.csv', ';')
df_general_measures = pd.read_csv('/content/gpu_data/GENERAL_MEASURES.csv', ';')
df_leg_for_lookup_codes = pd.read_csv('/content/gpu_data/LEGEND_FOR_LOOKUP_CODES.csv', ';')

joint_gm_names = ','.join(list(df_general_measures['MEASURE_TYPE'].astype(str).values_host))
joint_gm_values = ','.join(list(df_general_measures['MEASURE_VALUE'].astype(str).values_host))
joint_am_names = ','.join(list(df_additional_measures['MEASURE_TYPE'].astype(str).values_host))
joint_am_values = ','.join(list(df_additional_measures['MEASURE_VALUE'].astype(str).values_host))
joint_tw_names = ','.join(list(df_additional_tw_measures['MEASURE_TYPE'].astype(str).values_host))
joint_tw_values = ','.join(list(df_additional_tw_measures['MEASURE_VALUE'].astype(str).values_host))

df['general_measures_names'] = joint_gm_names
df['general_measures_values'] = joint_gm_values
df['additional_measures_names'] = joint_am_names
df['additional_measures_values'] = joint_am_values
df['additional_tw_measures_names'] = joint_tw_names
df['additional_tw_measures_values'] = joint_tw_values


def replace_formula_usd_kernel(row):
  replaced_formula = row['Formula_USD']

  general_measures_names = row['general_measures_names']
  general_measures_values = row['general_measures_values']
  additional_measures_names = row['additional_measures_names']
  additional_measures_values = row['additional_measures_values']
  additional_tw_measures_names = row['additional_tw_measures_names']
  additional_tw_measures_values = row['additional_tw_measures_values']

  arr_general_measures_names = []
  arr_general_measures_values = []
  arr_additional_measures_names = []
  arr_additional_measures_values = []
  arr_additional_tw_measures_names = []
  arr_additional_tw_measures_values = []

  first_comma_pos = 0
  last_comma_pos = 0

  first_comma_found = False
  last_comma_found = False

  len_gm = len(general_measures_names)
  len_adm = len(additional_measures_names)
  len_admtw = len(additional_tw_measures_names)

  count_gm = 0
  count_adm = 0
  count_admtw = 0


  # split general_measures_names
  while count_gm < len_gm:
    i = count_gm

    if general_measures_names[i] == ',':
      if not first_comma_found:
        first_comma_pos = i
        first_comma_found = True
      else:
        last_comma_pos = i
        last_comma_found =  True

    if first_comma_found and last_comma_found:
      piece = '@' + general_measures_names[first_comma_pos + 1:last_comma_pos - 1]
      arr_general_measures_names.append(piece)
      first_comma_found = False
      last_comma_found = False

    count_gm += 1


  # split additional_measures_names
  while count_adm < len_adm:
    i = count_adm

    if additional_measures_names[i] == ',':
      if not first_comma_found:
        first_comma_pos = i
        first_comma_found = True
      else:
        last_comma_pos = i
        last_comma_found =  True

    if first_comma_found and last_comma_found:
      piece = '@' + additional_measures_names[first_comma_pos + 1:last_comma_pos - 1]
      arr_additional_measures_names.append(piece)
      first_comma_found = False
      last_comma_found = False

    count_adm += 1


  # split additional_tw_measures_names
  while count_admtw < len_admtw:
    i = count_admtw

    if additional_tw_measures_names[i] == ',':
      if not first_comma_found:
        first_comma_pos = i
        first_comma_found = True
      else:
        last_comma_pos = i
        last_comma_found =  True

    if first_comma_found and last_comma_found:
      piece = '@' + additional_tw_measures_names[first_comma_pos + 1:last_comma_pos - 1]
      arr_additional_tw_measures_names.append(piece)
      first_comma_found = False
      last_comma_found = False

    count_admtw += 1


  count_gm = 0
  count_adm = 0
  count_admtw = 0

  #replace general_measures
  while count_gm < len_gm:
    i = count_gm

    if replaced_formula.contains(arr_general_measures_names[i]):
      replaced_formula = replaced_formula.replace(arr_additional_measures_names[i], arr_additional_measures_values[i])

    count_gm += 1


  #replace additional_measures_names
  while count_adm < len_adm:
    i = count_adm

    if replaced_formula.contains(arr_additional_measures_names[i]):
      replaced_formula = replaced_formula.replace(arr_additional_measures_names[i], arr_additional_measures_values[i])

    count_adm += 1

  #replace additional_tw_measures_names
  while count_admtw < len_admtw:
    i = count_admtw

    if replaced_formula.contains(arr_additional_tw_measures_names[i]):
      replaced_formula = replaced_formula.replace(arr_additional_tw_measures_names[i], arr_additional_tw_measures_values[i])

    count_admtw += 1


  return replaced_formula


df['Updated_Formula_USD'] = df.apply(replace_formula_usd_kernel, axis=1)

ValueError: user defined function compilation failed.